# Построение алгоритма классификации

## Команда Students:  
__Сивцов Данил__  
__Тыцкий Владислав__  

## Архитектура
#### Псевдолейблинг
1. Обучаем tfidf charwb + svm
2. Исключаем классы, которые svm очень плохо различает (71,84,204,79,83)
3. Берем метки с уверенностью больше 0.75
4. Произвольно выбираем из оставшихся 15000 и добаляем к изначальному датасету

Была также выделена небольшая валидационная выборка для тестирования доразметки. После подбора параметров стекинг обучался на всех данных

#### Далее работаем с полученными доразмеченными данными
#### Базовые модели
1. tfidf charwb + calibrated svm  
2. Numerical & categorical features + lgb  
3. tfidf char + naive bayes  

Взята часть категориальных фичей из публичного решения exotol create_features_part_2()

#### Стекинг - predict proba + svm

## Что не сработало:
1. KNN фичи на tfidf -> lgb (хотя в аналогичном соревновании от Яндекс https://www.youtube.com/watch?v=WSiPNCzShSY&t=691s&ab_channel=ODSAIRu они улучшили модель)
2. Стекинг бустингом. Как на классах, так и на вероятностях
3. Стекинг моделей с различным tfidf: charwb и word
4. Увеличение и уменьшение порога доразметки и количество добавляемых элементов
5. Агрегация фич по чекам (в соревновании Яндекса это улучшило скор)
6. Разделение camel case чеков, добавление к исходному названию склееного название чеков (для нас это оказалось странным, но паблик падал на пару тысячных, решили убрать)

## Что мы не успели попробовать/догадаться
1. Использовать Fast to text
2. Семплировать с различными сидами и усреднять несколько стекингов


### Подключаем поддержку GPU для lgb

In [ ]:
# After running
! git clone --recursive https://github.com/Microsoft/LightGBM

fatal: destination path 'LightGBM' already exists and is not an empty directory.


In [ ]:
#You can run this oneliner which will build and compile LightGBM with GPU enabled in colab:
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;  

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
Traceback (most recent call last):
  File "/usr/local/bin/cmake", line 8, in <module>
    sys.exit(cmake())
  File "/usr/local/lib/python2.7/dist-packages/cmake/__init__.py", line 34, in cmake
    raise SystemExit(_program('cmake', sys.argv[1:]))
  File "/usr/local/lib/python2.7/dist-packages/cmake/__init__.py", line 30, in _program
    return subprocess.call([os.path.join(CMAKE_BIN_DIR, name)] + args)
  File "/usr/lib/python2.7/subprocess.py", line 172, in call
    return Popen(*popenargs, **kwargs).wait()
  File "/usr/lib/python2.7/subprocess.py", line 1099, in wait
    pid, sts = _eintr_retry_call(os.waitpid, self.pid, 0)
  File "/usr/lib/python2.7/subprocess.py", line 125, in _eintr_retry_

#### После установки необходимо сделать restart runtime и начать с ячеек ниже

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import pickle
import re

import lightgbm as lgb

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, train_test_split, StratifiedKFold
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from scipy import sparse

from nltk.corpus import stopwords 

In [ ]:
!ls

drive  LightGBM  sample_data


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
stop_russian_words = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
path_to_data = "drive/MyDrive/Data Fusion/"

#### Загрузка выделенной выборки для тестирования алгоритмов на доразметке

In [ ]:
test = pd.read_parquet(path_to_data + "new_data_20_03/" + 'marked_test_agg.parquet') # все фичи уже саггрерированы!!

In [ ]:
test.shape

(8000, 11)

In [ ]:
test.head(5)

,receipt_dayofweek,item_quantity,item_price,item_nds_rate,day_of_part,is_int_quanitity,mean_item_quantity,mean_item_price,mean_item_counts,category_id,item_name
index,,,,,,,,,,,
30166,3,1.666667,9.000000,6,afternoon,False,1.303704,8.111111,6.000000,0,"Пив нап ""Вайс Берг пшеничное ЛИМОН-ЛАЙМ"""
35055,3,0.270769,15.923077,6,afternoon,True,0.760936,11.953219,5.769231,71,Салат Рыба ананасовая
28618,5,2.454545,8.795455,1,evening,False,1.664288,8.317610,4.814935,0,"ПИВО БАД СВЕТЛОЕ СТ/Б 5% 0,47Л"
16819,4,1.177083,8.041667,2,afternoon,False,1.152636,7.615305,6.260417,80,КАРТОФЕЛЬ ОВОЩАТА 1.5КГ
28262,5,1.162393,8.000000,1,afternoon,False,1.139615,8.575688,8.145299,19,Открытка О** Арт-Дизайн


#### Доразмеченные данные вместе с тестовыми (их необходимо позже исключить)

In [ ]:
parquet_train = pd.read_parquet(path_to_data + "new_data_20_03/" + 'all_marking_up_train.parquet') # как исходный датасет, но с разметкой

In [ ]:
parquet_train.head(5)

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,brands,category_id,category_id_prob
0,11,6,20:34,"Молоко 3,2%,шт",2.0,8,2,None,78,1.0
1,39,4,11:28,"Компот из изюма, 114 ккал",1.0,4,1,None,71,1.0
2,39,4,11:28,"Макаронные изделия отварные (масло сливочное),...",1.0,4,1,None,71,1.0
3,56,5,11:42,Кофе Капучино Большой Эден 18,1.0,12,1,None,70,1.0
4,105,3,01:53,Хлеб на СЫВОРОТКЕ 350г,1.0,7,-1,None,84,1.0


In [ ]:
# Выделение изначальной выборки
marked = parquet_train[parquet_train["category_id_prob"] == 1]

In [ ]:
forbidden_classes = [71,84,204,79,83]

In [ ]:
hold = 0.75
# Выделение доразмеченных данных и исключение смешивающихся классов
not_marked = parquet_train[((parquet_train["category_id_prob"] < 1) & (parquet_train["category_id_prob"] >= hold) &  
                            (~parquet_train["category_id"].isin(forbidden_classes)))]

In [ ]:
not_marked.shape

(7628613, 10)

In [ ]:
not_marked_no_dc = not_marked.drop_duplicates("item_name")

In [ ]:
size_not_marked = 15000
not_marked_sample = not_marked[not_marked["item_name"].isin(not_marked_no_dc["item_name"].sample(size_not_marked, random_state=42).values)]

In [ ]:
train_concat = pd.concat([marked, not_marked_sample], ignore_index=True)

In [ ]:
train_concat.shape, marked.shape, not_marked_sample.shape

((6580028, 10), (6483226, 10), (96802, 10))

#### Создание дополнительных признаков по чекам и по именам

In [ ]:
def custom_mode(x):
  return x.value_counts().index[0]

def check_is_int(x):
  return x.mean() !=1

In [ ]:
true_cheq = parquet_train['receipt_id'].unique()
X_train_cheq = parquet_train[parquet_train["receipt_id"].isin(true_cheq)]
X_train_cheq["item_counts"] = 1
X_train_by_reciept = X_train_cheq.groupby("receipt_id").agg({
    "item_quantity": "mean",
    "item_price": "mean",
    "item_counts": "sum"
    })

In [ ]:
X_train_by_reciept["receipt_id"] = X_train_by_reciept.index
X_train_by_reciept.reset_index(drop=True, inplace=True)
X_train_by_reciept.rename(columns={"item_quantity":"mean_item_quantity","item_price":"mean_item_price",	"item_counts":"mean_item_counts"},inplace=True)

In [ ]:
X_train_by_reciept.shape

(8127608, 4)

In [ ]:
X_train_base = train_concat.reset_index(drop=True)

In [ ]:
X_train_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6580028 entries, 0 to 6580027
Data columns (total 10 columns):
 #   Column             Dtype  
---  ------             -----  
 0   receipt_id         int64  
 1   receipt_dayofweek  int32  
 2   receipt_time       object 
 3   item_name          object 
 4   item_quantity      float64
 5   item_price         int64  
 6   item_nds_rate      int32  
 7   brands             object 
 8   category_id        int64  
 9   category_id_prob   float64
dtypes: float64(2), int32(2), int64(3), object(3)
memory usage: 451.8+ MB


In [ ]:
def part_of_day(x):
    if ((x > 23) or x>=0) and (x <= 7):
        return 'night'
    elif (x > 7) and (x <= 12):
        return 'morning'
    elif (x > 12) and (x <= 17):
        return 'afternoon'
    elif (x > 17) and (x <= 20) :
        return 'evening'
    elif (x > 20) and (x <= 23):
        return 'late evening'


In [ ]:
X_train_base["day_of_part"] = X_train_base["receipt_time"].apply(lambda x: part_of_day(int(x[:2])))

In [ ]:
def type_of_quan(x):
  if(int(x) == x):
    return True
  return False

In [ ]:
X_train_base["is_int_quanitity"] =  X_train_base["item_quantity"].apply(lambda x: type_of_quan(x))

In [ ]:
X_train_base = X_train_base.merge(X_train_by_reciept, on="receipt_id", how="left")

In [ ]:
X_train_base.head(7)

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,brands,category_id,category_id_prob,day_of_part,is_int_quanitity,mean_item_quantity,mean_item_price,mean_item_counts
0,11,6,20:34,"Молоко 3,2%,шт",2.0,8,2,None,78,1.0,evening,True,2.000000,8.000000,1
1,39,4,11:28,"Компот из изюма, 114 ккал",1.0,4,1,None,71,1.0,morning,True,1.000000,5.250000,4
2,39,4,11:28,"Макаронные изделия отварные (масло сливочное),...",1.0,4,1,None,71,1.0,morning,True,1.000000,5.250000,4
3,56,5,11:42,Кофе Капучино Большой Эден 18,1.0,12,1,None,70,1.0,morning,True,1.000000,11.000000,2
4,105,3,01:53,Хлеб на СЫВОРОТКЕ 350г,1.0,7,-1,None,84,1.0,night,True,1.000000,7.000000,1
5,122,0,11:46,Сосиска в тесте с сыром 1шт ГЕ,3.0,7,2,None,84,1.0,morning,True,1.562364,8.818182,11
6,129,3,15:17,ЛанчБаскет 5 за 300: 2 шт ОРИГ Стрипсы кур фил,1.0,8,2,None,69,1.0,afternoon,True,1.166667,8.166667,6


In [ ]:
X_all_agg = X_train_base.groupby("item_name").agg({
    "receipt_dayofweek":custom_mode,
    "item_quantity":"mean",
    "item_price":"mean",
    "item_nds_rate":custom_mode,
    "day_of_part":custom_mode,
    "is_int_quanitity": check_is_int,
    "mean_item_quantity": "mean",
    "mean_item_price": "mean",
    "mean_item_counts": "mean",
    "category_id":custom_mode,
    "category_id_prob":"mean",
    })

In [ ]:
# Фичи от exotol
def create_features_part_2(data):
    data['is_preservativ'] = data.item_name.apply(lambda x: int('презер' in x.lower()))
    data['is_auto'] = data.item_name.apply(lambda x: int('авто' in x.lower() or \
                                                            'щетк' in x.lower() or \
                                                             'сантиметр' in x.lower()))

    data['is_paper'] = data.item_name.apply(lambda x: int('бумага' in x.lower()))
    data['is_otkritka'] = data.item_name.apply(lambda x: int('открытка' in x.lower()))
    data['is_born'] = data.item_name.apply(lambda x: int('рождения' in x.lower()))
    data['is_a4'] = data.item_name.apply(lambda x: int('а4' in x.lower()))
    data['is_ubiley'] = data.item_name.apply(lambda x: int('юбиле' in x.lower()))

    data['is_list'] = data.item_name.apply(lambda x: int('лист' in x.lower()))
    data['is_kanc'] = data.item_name.apply(lambda x: int('канц' in x.lower()))
    data['is_dnevnik'] = data.item_name.apply(lambda x: int('дневник' in x.lower()))

    data['item_name_len'] = data.item_name.apply(lambda x: len(x.split()))

    return data

In [ ]:
X_all_agg["item_name"] = X_all_agg.index
X_all_agg.reset_index(drop=True, inplace=True)# НА ЭТОМ НАДО ОБУЧАТЬ ФИНАЛЬНУЮ МОДЕЛЬ

In [ ]:
X_all_agg = create_features_part_2(X_all_agg)

In [ ]:
#X_train_agg['item_name'] = X_train_agg['item_name'].apply(lambda x: re.sub(r'([а-я])([А-Я]{1,2})', r'\1 \2', x) + ' ' + re.sub(r' ', r'', x))

In [ ]:
X_all_agg.head(5)

,receipt_dayofweek,item_quantity,item_price,item_nds_rate,day_of_part,is_int_quanitity,mean_item_quantity,mean_item_price,mean_item_counts,category_id,category_id_prob,item_name,is_preservativ,is_auto,is_paper,is_otkritka,is_born,is_a4,is_ubiley,is_list,is_kanc,is_dnevnik,item_name_len
0,4,0.702276,11.775862,6,afternoon,True,1.005162,10.519226,4.137931,84,1.0,,0,0,0,0,0,0,0,0,0,0,0
1,6,2.375333,5.333333,5,afternoon,True,1.376065,7.561111,5.166667,80,1.0,КАРТОФЕЛЬ ВЕС,0,0,0,0,0,0,0,0,0,0,2
2,6,1.637363,9.000000,1,afternoon,False,1.398866,7.828059,5.318681,130,1.0,МИСКА,0,0,0,0,0,0,0,0,0,0,1
3,6,1.222222,9.000000,-1,afternoon,False,1.264021,8.005820,3.777778,61,1.0,НОСКИ,0,0,0,0,0,0,0,0,0,0,1
4,5,0.642447,9.212766,5,afternoon,True,0.835412,9.079838,4.042553,80,1.0,БАНАНЫ кг,0,0,0,0,0,0,0,0,0,0,2


In [ ]:
X_train_agg = X_all_agg[~X_all_agg["item_name"].isin(test["item_name"].values)] # обязательно убрать из трейна тестовые объекты!!!

In [ ]:
X_train_agg.shape

(55216, 23)

### Промежуточное сохранение/загрузка доразмеченных данных

In [ ]:
#X_train_agg.to_csv(path_to_data+"stacking_19_03_vlad"+"x_train_agg_with_not_marked.csv")

In [ ]:
X_train_agg = pd.read_csv(path_to_data+"stacking_19_03_vlad"+"x_train_agg_with_not_marked.csv", index_col='Unnamed: 0')

In [ ]:
X_train_agg = X_train_agg.iloc[1:]

In [ ]:
X_train_agg.reset_index(drop=True, inplace=True)

In [ ]:
X_train_agg

## TFIDF

In [ ]:
X_train_agg.shape

(63216, 23)

In [ ]:
X_all_agg.shape

(63216, 23)

In [ ]:
X_train_agg = X_all_agg

In [ ]:
tfidf = TfidfVectorizer(stop_words=stop_russian_words, max_features=160000, ngram_range=(1,5), analyzer="char_wb")
X_train_tfidf = tfidf.fit_transform(X_train_agg["item_name"])
X_test_tfidf =  tfidf.transform(test["item_name"])

In [ ]:
tfidf_prod = TfidfVectorizer(stop_words=stop_russian_words, max_features=160000, ngram_range=(1,5), analyzer="char_wb")
X_train_tfidf_prod = tfidf_prod.fit_transform(X_all_agg["item_name"])

pickle.dump(tfidf_prod, open(path_to_data+'/stacking_21_03/'+'tfidf', 'wb'))

In [ ]:
X_all_agg

,receipt_dayofweek,item_quantity,item_price,item_nds_rate,day_of_part,is_int_quanitity,mean_item_quantity,mean_item_price,mean_item_counts,category_id,category_id_prob,item_name,is_preservativ,is_auto,is_paper,is_otkritka,is_born,is_a4,is_ubiley,is_list,is_kanc,is_dnevnik,item_name_len
0,4,0.702276,11.775862,6,afternoon,True,1.005162,10.519226,4.137931,84,1.000000,,0,0,0,0,0,0,0,0,0,0,0
1,6,2.375333,5.333333,5,afternoon,True,1.376065,7.561111,5.166667,80,1.000000,КАРТОФЕЛЬ ВЕС,0,0,0,0,0,0,0,0,0,0,2
2,6,1.637363,9.000000,1,afternoon,False,1.398866,7.828059,5.318681,130,1.000000,МИСКА,0,0,0,0,0,0,0,0,0,0,1
3,6,1.222222,9.000000,-1,afternoon,False,1.264021,8.005820,3.777778,61,1.000000,НОСКИ,0,0,0,0,0,0,0,0,0,0,1
4,5,0.642447,9.212766,5,afternoon,True,0.835412,9.079838,4.042553,80,1.000000,БАНАНЫ кг,0,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63211,3,1.000000,10.000000,6,afternoon,False,1.000000,12.333333,3.000000,42,0.808911,"№07 Step Лак для ногтей ""Dance Legend"" шт",0,0,0,0,0,0,0,0,0,0,8
63212,1,1.509434,11.000000,6,afternoon,False,1.401258,10.004403,2.207547,71,1.000000,№1 с люля-кебаб (фри) 1 порц,0,0,0,0,0,0,0,0,0,0,6
63213,4,1.000000,9.000000,6,late evening,False,1.500000,9.000000,2.000000,67,0.796135,"№3 Суши креветка ""Эби""",0,0,0,0,0,0,0,0,0,0,4
63214,2,1.000000,18.000000,6,late evening,False,1.000000,18.000000,1.000000,67,0.877610,№5 Сет-Филадельфия,0,0,0,0,0,0,0,0,0,0,2


## SVM над TFIDF

In [ ]:
params = {
      "max_iter" : 2000, 
      "penalty" : "l2",
      "C" : 0.9,
      "tol" : 1e-6,
      "random_state" : 42,
}

calib_model = CalibratedClassifierCV(LinearSVC(**params))
calib_model.fit(X_train_tfidf, X_train_agg["category_id"]) # ТРЕТИЙ ПАРАМЕТР МОЖНО ДОБАВИТЬ (НУЖНЫ ЭКСПЕРИМЕНТЫ)

CalibratedClassifierCV(base_estimator=LinearSVC(C=0.8, class_weight=None,
                                                dual=True, fit_intercept=True,
                                                intercept_scaling=1,
                                                loss='squared_hinge',
                                                max_iter=2000,
                                                multi_class='ovr', penalty='l2',
                                                random_state=42, tol=1e-06,
                                                verbose=0),
                       cv=None, method='sigmoid')

In [ ]:
y_pred = calib_model.predict(X_test_tfidf)
f1_score(y_pred, test["category_id"], average="weighted")
# 0.8469366334321011 без дорахметки
# 0.8484153311883393
# 0.8488989894744159 balanced
# 0.848911177179441 no balanced
# 0.849173317346558 no bad classes + balances
# 0.8477197001215887 0.85 + no bad classes + balances
# 0.8492310007819597 0.75 + no bad classes + balances
# 0.8470262437296944 0.75 + no bad classes (more) + balances
# 0.8480243236659688 0.75 + no bad classes (less) + balances
# 0.8493783343503385 0.75 + no bad classes (less) + balances + 115000
# 0.8496948644974475 0.75 + no bad classes (less) + balances + 110000
# 0.8491 ........... 0.75 + no bad classes (less) + balances + 105000
# 0.8487785299951248 0.8
# 0.8487709884622684 0.75 + 12000
# 0.8507260896737594 0.75 + 8000
# 0.8500041946390465 0.75 + 6000
# 0.8484266232970054 0.75 + 7000
# 0.849915817325545  0.75 + 9000

0.847754322774402

In [ ]:
params = {
      "max_iter" : 2000, 
      "penalty" : "l2",
      "C" : 0.9,
      "tol" : 1e-7,
      "random_state" : 42,
}

calib_model = CalibratedClassifierCV(LinearSVC(**params))
calib_model.fit(X_train_tfidf, X_train_agg["category_id"])
print("Калибровка закончена")

kfold = StratifiedKFold(n_splits=5, random_state=41, shuffle=True)  
cv = kfold.split(X_train_tfidf, X_train_agg["category_id"])

y_pred_svm = cross_val_predict(calib_model, X_train_tfidf, X_train_agg["category_id"], cv=cv, method="predict_proba", n_jobs=-1)

Калибровка закончена


In [ ]:
X_train_tfidf_prod

<63216x160000 sparse matrix of type '<class 'numpy.float64'>'
	with 7056109 stored elements in Compressed Sparse Row format>

In [ ]:
# SAVE MODEL

params = {
      "max_iter" : 2000, 
      "penalty" : "l2",
      "C" : 0.9,
      "tol" : 1e-7,
      "random_state" : 42,
}

model_svm_p_prod = LinearSVC(**params)

calib_model_p_prod = CalibratedClassifierCV(model_svm_p_prod)
calib_model_p_prod.fit(X_train_tfidf_prod, X_all_agg["category_id"])

pickle.dump(calib_model_p_prod, open(path_to_data+'/stacking_21_03/'+'svm_tfidf', 'wb'))

## GB над categorical and numerical features+preprocessing

In [ ]:
numeric_features = ['item_quantity', 'item_price', 'mean_item_quantity', 'mean_item_price', 'mean_item_counts']
categorical_features = ['receipt_dayofweek', 'item_nds_rate', 'day_of_part']
binary_features = ['is_int_quanitity', 'is_preservativ', 'is_paper', 'is_otkritka', 'is_born', 'is_a4', 'is_ubiley',                                         
                 'is_auto', 'is_list', 'is_kanc', 'is_dnevnik']

In [ ]:
ohe = OneHotEncoder(handle_unknown="ignore")
sd = StandardScaler()
X_cat = ohe.fit_transform(X_train_agg[categorical_features])
X_num = sd.fit_transform(X_train_agg[numeric_features])
X_bi = X_train_agg[binary_features].values

In [ ]:
X_num_cat = np.hstack([X_num, X_cat.toarray(), X_bi])

In [ ]:
# SAVE MODEL
pickle.dump(ohe, open(path_to_data+'/stacking_20_03_markup/'+'prepro_ohe', 'wb'))
pickle.dump(sd, open(path_to_data+'/stacking_20_03_markup/'+'prepro_sd', 'wb'))

In [ ]:
params = {
    "n_estimators" : 150,
    "objective" : "multiclass",
    "metric" : "multi_logloss",
    "num_class" : 96,
    "reg_lambda" : 1,
    "n_jobs":-1
}

model_num_cat = lgb.LGBMClassifier(**params)
model_num_cat.fit(X_num_cat, X_train_agg["category_id"])

In [ ]:
y_pred = model_num_cat.predict(test) #Надо тест трансформировать
f1_score(y_pred, test["category_id"])

In [ ]:
params = {
    "n_estimators" : 110,
    "objective" : "multiclass",
    "metric" : "multi_logloss", # можно поменять
    "num_class" : 96,
    "reg_lambda" : 1,
    "n_jobs" : -1
}

model_num_cat = lgb.LGBMClassifier(**params)

kfold = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)  
cv = kfold.split(X_num_cat, X_train_agg["category_id"])

y_pred_num_cat = cross_val_predict(model_num_cat, X_num_cat, X_train_agg["category_id"], cv=cv, method="predict_proba", n_jobs=-1)

#### prod

In [ ]:
ohe_prod = OneHotEncoder(handle_unknown="ignore")
sd_prod = StandardScaler()
X_cat_prod = ohe_prod.fit_transform(X_all_agg[categorical_features])
X_num_prod = sd_prod.fit_transform(X_all_agg[numeric_features])
X_bi_prod = X_all_agg[binary_features].values

In [ ]:
X_num_cat_prod = np.hstack([X_num_prod, X_cat_prod.toarray(), X_bi_prod])

In [ ]:
# SAVE MODEL
pickle.dump(ohe_prod, open(path_to_data+'/stacking_21_03/'+'prepro_ohe', 'wb'))
pickle.dump(sd_prod, open(path_to_data+'/stacking_21_03/'+'prepro_sd', 'wb'))

In [ ]:
# SAVE MODEL
params = {
    "n_estimators" : 110,
    "objective" : "multiclass",
    "metric" : "multi_logloss", # можно поменять
    "num_class" : 96,
    "reg_lambda" : 1,
    "n_jobs" : -1
}

model_num_cat_p_prod = lgb.LGBMClassifier(**params)
model_num_cat_p_prod.fit(X_num_cat_prod, X_all_agg["category_id"])

pickle.dump(model_num_cat_p_prod, open(path_to_data+'/stacking_21_03/'+'gb_cat_num', 'wb'))

## NaiveBayes

In [ ]:
nb = MultinomialNB(alpha=0.02)
nb.fit(X_train_tfidf, X_train_agg["category_id"])

MultinomialNB(alpha=0.02, class_prior=None, fit_prior=True)

In [ ]:
y_pred = nb.predict(X_test_tfidf)
f1_score(y_pred, test["category_id"], average="weighted")
# 0.813311980298414

0.8134524306128547

In [ ]:
model_nb = MultinomialNB(alpha=0.003)

kfold = StratifiedKFold(n_splits=7, random_state=40, shuffle=True)  
cv = kfold.split(X_train_tfidf, X_train_agg["category_id"])

y_pred_nb = cross_val_predict(model_nb, X_train_tfidf, X_train_agg["category_id"], cv=cv, method="predict_proba", n_jobs=-1)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [ ]:
# SAVE MODEL
model_nb_p_prod = MultinomialNB(alpha=0.003)
model_nb_p_prod.fit(X_train_tfidf_prod, X_all_agg["category_id"])

pickle.dump(model_nb_p_prod, open(path_to_data+'/stacking_21_03/'+'nb_tfdif', 'wb'))

## Stacking

In [ ]:
X_train_stacking = np.hstack((y_pred_svm, y_pred_num_cat, y_pred_nb))

In [ ]:
X_train_stacking.shape

(48220, 288)

In [ ]:
params = {
      "max_iter" : 2000, 
      "penalty" : "l2",
      "C" : 1,
      "tol" : 1e-6,
      "random_state" : 42,
}

model_svm_stacking = LinearSVC(**params,)

kfold = StratifiedKFold(n_splits=5, random_state=39, shuffle=True)  
cv = kfold.split(X_train_stacking, X_train_agg["category_id"])

cv_score = cross_val_score(model_svm_stacking, X_train_stacking, X_train_agg["category_id"], cv=cv, scoring='f1_weighted', n_jobs=-1)
cv_score, cv_score.mean()
# (array([0.85371468, 0.84346967, 0.85472305, 0.84798714, 0.8439762 ]),
# 0.8487741484570266)
# (array([0.85705507, 0.8468878 , 0.85598468, 0.84854355, 0.84674792]),
# 0.8510438057445076)
# (array([0.87338505, 0.87668755, 0.8720676 , 0.87894466, 0.87103098]),
# 0.8744231679054044) no balance + balance
# (array([0.87339161, 0.87667962, 0.87206957, 0.87913748, 0.87113532]),
# 0.8744827189212951) no balance + no balance
# (array([0.87415502, 0.87401062, 0.87457548, 0.87610206, 0.87652818]),
# 0.8750742710101029) no forbidden + no balance + no balance
# (array([0.87409408, 0.87739453, 0.8764946 , 0.87854043, 0.87925427]),
# 0.8771555841600961) 0.75 + no forbidden + no balance + no balance
# (array([0.87675724, 0.8732235 , 0.87709711, 0.88472741, 0.87986508]),
# 0.8783340680380063)

# (array([0.87069333, 0.86666485, 0.87051296, 0.87871016, 0.86615952]),
# 0.8705481645253952) 0.75 + no forbidden + no balance + no balance + 8000 + 110000
# (array([0.8728736 , 0.86755725, 0.87068626, 0.88031831, 0.86672393]),
# 0.8716318689603838) c=0.8



# (array([0.87526649, 0.87717083, 0.87639529, 0.87933725, 0.87958285]),
# 0.8775505394752423) 0.001

(array([0.89086481, 0.88430327, 0.88618538, 0.88965636, 0.89216889]),
 0.8886357443397991)

In [ ]:
(array([0.88955504, 0.88749026, 0.88632709, 0.89287237, 0.89075067]),
 0.8893990849788898)

In [ ]:
(array([0.87436203, 0.87735661, 0.87612731, 0.8796682 , 0.87982066]),
 0.8774669625514709)

### Сохраним итоговою модель

In [ ]:
# SAVE MODEL
params = {
      "max_iter" : 2500, 
      "penalty" : "l2",
      "C" : 1.05,
      "tol" : 1e-8,
      "random_state" : 40,
}
model_svm_stacking_p = LinearSVC(**params)

model_svm_stacking_p.fit(X_train_stacking, X_train_agg["category_id"]) # ТРЕТИЙ ПАРАМЕТР МОЖНО УБРАТЬ(НУЖНЫ ЭКСПЕРИМЕНТЫ)

pickle.dump(model_svm_stacking_p, open(path_to_data+'/stacking_21_03/'+'svm_stacking', 'wb'))